### 방문자 분석

##### 라이브러리

In [139]:
# 라이브러리
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import mannwhitneyu, shapiro

##### 데이터 불러오기 및 전처리

In [138]:
visitor_df = pd.read_csv('./data/visitor_df.csv')
visitor_df['date'] = pd.to_datetime(visitor_df['date'])
visitor_df['year_month'] = visitor_df['date'].dt.to_period('M').astype(str)

# 개선일 설정
improvement_date = pd.to_datetime('2024-05-23')

##### 1. 월별 방문자 수 집계 및 시각화

In [142]:
# 월별 방문자 수 집계
monthly_visitors = visitor_df.groupby('year_month')['visitor'].sum().reset_index()

# 월별 방문자 수 그래프 (Plotly)
fig = px.bar(
    monthly_visitors, x='year_month', y='visitor',
    title='월별 방문자 수', labels={'year_month': '연도-월', 'visitor': '방문자 수'},
    template='plotly_white'
)
fig.update_layout(xaxis_title='연도-월', xaxis_tickvals=monthly_visitors['year_month'],
                  xaxis_ticktext=monthly_visitors['year_month'].astype(str), xaxis_tickangle=-45)
fig.show()

##### 2. 방문자 및 페이지뷰 집계 및 시각화

In [147]:
monthly_average_visitors = visitor_df.groupby('year_month')['visitor'].mean().reset_index()
monthly_average_visitors.columns = ['year_month', 'average_visitor']

# 월별 평균 방문자 수 그래프
fig = px.bar(
    monthly_average_visitors, 
    x='year_month', y='average_visitor',
    title='월별 평균 방문자 수', 
    labels={'year_month': '연도-월', 'average_visitor': '방문자 수 (평균)'},
    template='plotly_white'
)
fig.update_layout(xaxis_title='연도-월', xaxis_tickangle=-45)
fig.show()


##### 3. 방문자 및 페이지뷰 집계 및 시각화

In [143]:
monthly_data = visitor_df.groupby('year_month').agg(
    monthly_visitors=('visitor', 'sum'),
    monthly_views=('view', 'sum')
).reset_index()
monthly_data['avg_pageviews_per_visitor'] = monthly_data['monthly_views'] / monthly_data['monthly_visitors']

# 방문자와 페이지뷰 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_data['year_month'], y=monthly_data['monthly_visitors'],
                         mode='lines+markers', name='Visitors', line=dict(color='skyblue')))
fig.add_trace(go.Scatter(x=monthly_data['year_month'], y=monthly_data['monthly_views'],
                         mode='lines+markers', name='Page Views', line=dict(color='blue', dash='dash')))
fig.update_layout(title='월별 방문자 수와 페이지뷰 비교',
                  xaxis=dict(title='Year-Month', tickangle=-45),
                  yaxis=dict(title='Visitors'),
                  yaxis2=dict(title='Page Views', overlaying='y', side='right'),
                  template='plotly_white')
fig.show()


##### 4. 개선 전후 방문자 수 분석

In [157]:
visitor_df['improvement'] = visitor_df['date'] >= improvement_date

def get_pre_post_data(days):
    pre_data = visitor_df[(visitor_df['date'] < improvement_date) & 
                          (visitor_df['date'] >= improvement_date - pd.DateOffset(days=days))]
    post_data = visitor_df[(visitor_df['date'] >= improvement_date) & 
                           (visitor_df['date'] < improvement_date + pd.DateOffset(days=days))]
    return pre_data['visitor'], post_data['visitor']

def perform_analysis(days):
    pre_visitors, post_visitors = get_pre_post_data(days)
    pre_shapiro = shapiro(pre_visitors).pvalue
    post_shapiro = shapiro(post_visitors).pvalue

    print(f"{days}일 기준:")
    print(f"개선 전 정규성 p-value: {pre_shapiro:.5f}")
    print(f"개선 후 정규성 p-value: {post_shapiro:.5f}")

    if pre_shapiro > 0.05 and post_shapiro > 0.05:
        print("정규성이 만족하므로 T-test를 사용해야 합니다. 그러나 비모수 검정으로 진행합니다.")
    else:
        print("정규성 불만족으로 비모수 검정 사용.")
        stat, p_value = mannwhitneyu(pre_visitors, post_visitors)
        print(f"Mann-Whitney U 결과: U-statistic = {stat:.2f}, p-value = {p_value:.5f}")
        print("웹사이트 개선이 방문자 수 증가에 유의미한 영향을 미쳤습니다." if p_value < 0.05 else "방문자 수 변화가 유의미하지 않습니다.")
        print(" ")

##### 5. 30일 및 90일 분석 실행

In [160]:
# 30일 및 90일 분석 실행
perform_analysis(30)
perform_analysis(90)

# 개선 전후 평균 방문자 수 계산
pre_avg = visitor_df[visitor_df['improvement'] == False]['visitor'].mean()
post_avg = visitor_df[visitor_df['improvement'] == True]['visitor'].mean()
print(f"개선 전 평균 방문자 수: {pre_avg:.2f}")
print(f"개선 후 평균 방문자 수: {post_avg:.2f}")

30일 기준:
개선 전 정규성 p-value: 0.00013
개선 후 정규성 p-value: 0.00059
정규성 불만족으로 비모수 검정 사용.
Mann-Whitney U 결과: U-statistic = 185.00, p-value = 0.00009
웹사이트 개선이 방문자 수 증가에 유의미한 영향을 미쳤습니다.
 
90일 기준:
개선 전 정규성 p-value: 0.00000
개선 후 정규성 p-value: 0.00002
정규성 불만족으로 비모수 검정 사용.
Mann-Whitney U 결과: U-statistic = 552.00, p-value = 0.00000
웹사이트 개선이 방문자 수 증가에 유의미한 영향을 미쳤습니다.
 
개선 전 평균 방문자 수: 52.29
개선 후 평균 방문자 수: 118.95


##### 6. 개선 후 방문자 수 증가량 및 비율

In [179]:
# 개선 전후 방문자 수 비교를 위한 구분 열 추가
visitor_df['improvement'] = visitor_df['date'] >= improvement_date

# 개선 전 평균 방문자 수 계산
pre_avg = visitor_df[visitor_df['date'] < improvement_date]['visitor'].mean()

# 개선 후 평균 방문자 수 계산 함수
def calculate_post_avg(period_months):
    post_period = visitor_df[(visitor_df['date'] >= improvement_date) & 
                             (visitor_df['date'] < improvement_date + pd.DateOffset(months=period_months))]
    post_avg = post_period['visitor'].mean()
    return post_avg

# 각 기간(1개월, 3개월, 5개월) 비교
for months in [1, 3, 5]:
    post_avg = calculate_post_avg(months)
    
    # 증가량 및 비율 계산
    increase_amount = post_avg - pre_avg
    increase_rate = (increase_amount / pre_avg * 100) if pre_avg > 0 else None

    # 결과 출력
    print(f"\n개선 후 {months}개월 동안:")
    print(f"개선 전 평균 방문자 수: {pre_avg:.2f}")
    print(f"개선 후 평균 방문자 수: {post_avg:.2f}")
    print(f"증가량: {increase_amount:.2f}")
    print(f"증가 비율: {increase_rate:.2f}%" if increase_rate is not None else "증가 비율: 데이터 없음")



개선 후 1개월 동안:
개선 전 평균 방문자 수: 52.29
개선 후 평균 방문자 수: 121.06
증가량: 68.78
증가 비율: 131.53%

개선 후 3개월 동안:
개선 전 평균 방문자 수: 52.29
개선 후 평균 방문자 수: 137.95
증가량: 85.66
증가 비율: 163.81%

개선 후 5개월 동안:
개선 전 평균 방문자 수: 52.29
개선 후 평균 방문자 수: 118.95
증가량: 66.66
증가 비율: 127.47%
